In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel("https://www.sharkattackfile.net/spreadsheets/GSAF5.xls")

In [ ]:
import os

In [ ]:
run Shark_Quest.py

In [ ]:
to_use.nunique()


In [ ]:
data_1 = df.copy() #making first copy just in case

# making a new data set with columns that we want to work with instead of dropping columns

Here's how and logic: To select multiple columns, use a list of column names within the selection brackets
    The inner square brackets define a Python list with column names,
    whereas the outer brackets are used to select the data from a pandas DataFrame.
    The returned data type is a pandas DataFrame. This method does not affect the rows. so we retain original rows

# justification for keeping columns

    1)data: there might be a particular season for attacks -we want to avoid that
    2)year: we may look at this long term or select a period of time later - say 10 years
    3)type: as discussed - unprovoked attacks might signal how dangerous the shark type is
    4)country and location: we want to circle the hotspots
    5)injury: not sure but maybe fatal ones could be useful
    6)species: to find which ones require conservation
    # age and sex and all that - not required

In [ ]:
to_use = data_1[["Year","Type","Country","Location","Injury","Species "]].copy()

In [ ]:
#next step is data cleaning and standardisation - column names require cleaning - "species "has a white space
to_use.columns = to_use.columns.str.replace(' ', '')
to_use.columns

In [ ]:
to_use.columns = to_use.columns.str.lstrip().str.rstrip()
to_use.columns = to_use.columns.str.lower()

In [ ]:
to_use.isna().sum()

In [ ]:
to_use.dtypes #have to clear nans or have a strategy to clear nans before casting types i suppose?

In [ ]:
to_use.duplicated().sum() #check for duplicates

In [ ]:
#drop the duplicates
to_use = to_use.drop_duplicates()

In [ ]:
#checking for empty spaces
to_use.eq(" ").sum()

# tasks to do:
    - cleaning data set based on nans and what remains
    - duplicates
    - standardisation of column values
    - then value counts

# cleaning

In [ ]:
# year column

to_use['year'] = to_use['year'].astype(str).str.replace(".0", "")
to_use['year'] = to_use['year'].replace('nan', '0')
to_use['year'] = to_use['year'].astype(int)
to_use = to_use[to_use["year"] >= 2000] #we sorted the years starting in 2000 till the actual year because that is what we want to analyze
to_use = to_use.sort_values(by="year", ascending=True)
to_use = to_use[to_use["year"] < 2025]

In [ ]:
#column type
unwanted= ["Invalid", "Watercraft", "Questionable", "Sea Disaster", "Under investigation", "Unverified","Unconfirmed", "Boat", "?"]
to_use= to_use[to_use["type"].apply(lambda x: x not in unwanted)]


In [ ]:
to_use["type"] = to_use["type"].replace(" Provoked", "Provoked")
to_use["type"].value_counts()

In [ ]:
to_use = to_use.dropna(subset=["type"])
to_use.isna().sum()

In [ ]:
#checking for duplicates
to_use.duplicated().sum()
to_use.drop_duplicates(inplace=True)

In [ ]:
#finding nan values in year, location, country
to_use["location"].isna().sum()
to_use["country"].isna().sum()
to_use["year"].isna().sum()   #i checked and there were none in year

In [ ]:
#dropping nan values in country and location
to_use.dropna(subset = ['country', 'location'],inplace = True)

In [ ]:
# categories of injury

injuries_1 = ["no injury", "rammed", "knocked"]
injuries_2 = ["lacer"]
injuries_3 = ["minor", "bitten", "injured", "puncture", "superficial", "cut", "pinched", "bruised", "gash",
             "mark", "struck", "torn"]
injuries_4 = ["major", "sever", "significant", "broken", "injuries", "injury", "multiple", "serious",
              "gashed", "avulsed", "large", "lost", "amputated"]
injuries_5 = ["fatal", "death"]
boat = ["boat", "hull", "dinghy", "kayak"]
scavenging = ["scavenging"]

all_injuries = injuries_1 + injuries_2 + injuries_3 + injuries_4 + injuries_5 + boat + scavenging

def injury_rating(j):

    """categorising entries in the injury column"""

    j = j.lower()
    for i in injuries_1:
        if i in j:
            return "no injury"
    for i in injuries_2:
        if i in j:
            return "lacerations"
    for i in injuries_3:
        if i in j:
            return "minor injuries"
    for i in injuries_4:
        if i in j:
            return "major injuries"
    for i in injuries_5:
        if i in j:
            return "fatal"
    for i in scavenging:
        if i in j:
            return "probably scavenging"
    for i in boat:
        if i in j:
            return "material damage"
    

In [ ]:
to_use["injury"].unique()

In [ ]:
# categorising injuries
to_use.dropna(subset="injury", inplace=True)
to_use["injury"] = to_use["injury"].apply(injury_rating)
to_use.dropna(subset="injury", inplace=True)

In [ ]:
# categorising shark species

sharks = ["white shark", "tiger shark", "bull shark", "shortfin mako shark",
         "lemon shark", "oceanic whitetip shark", "blue shark", "galapagos shark", "caribbean reef shark",
         "dusky shark", "blacktip shark", "silky shark", "gray reef shark", "great hammerhead shark",
         "blacktip reef shark", "sevengill shark", "sixgill shark", "nurse shark",
         "sand tiger", "spotted wobbegong", "basking shark", "spinner shark", "bronze whaler", "blue pointer"]

def shark_species(value, sharks):
    for shark in sharks:
        if shark in value.lower():
            return shark

In [ ]:
to_use.dropna(subset="species", inplace=True)
to_use["species"] = to_use["species"].apply(shark_species, args=(sharks,))
to_use["species"] = to_use["species"].replace("blue pointer", "shortfin mako shark")
to_use.dropna(subset="species", inplace=True)

In [ ]:
#top ten locations - not sure if we need this
n=10
to_use["location"].value_counts()[:n].index.tolist()

In [ ]:
#top ten coutries - again not sure if we ened this
#south africa is however third
to_use["country"].value_counts()[:n].index.tolist()

# codes for cleaning locations  - odi 

- i took the longest to decide what to do with the long county names.
- option 1:  to group different places within the same county under the same county
- option 2: to leave the county names are often preceded by very specific areas in the county (counties exist mainly in the US)
  
      does this seem correct to you all?
- option 3: when you look at the dictionary returned against location and type (location_frequencies = to_use["location"][to_use["type"] == "Provoked"].value_counts().to_dict() # but this one returns a dictionary ) it gives different areas under the same county and their value count - now do you think if we group all areas under their common county names the result will change? and do we n

In [ ]:

#cleaning location
to_use.dropna(subset = ['location'],inplace = True)
to_use["location"].isna().sum()


In [ ]:
#removing numbers and special characters
to_use['location'] = to_use['location'].str.replace(r'[^a-zA-Z\s]', '', regex=True)

In [ ]:
#removing spaces
to_use['location'] = to_use['location'].str.strip().str.replace(r'\s+', ' ', regex=True)

# cleaning country  - odi


In [ ]:
#cleaning country
to_use['country'] = to_use['country'].str.title()

In [ ]:
#cleaning white spaces 
to_use["country"] = to_use['country'].str.strip()
to_use.country.to_list()

# NEW CODES PLEASE READ THE NEW REASONING BEHIND THIS


In [ ]:
#to get all the values corresponding to provoked
#this is so far okay - we are going for provoked locations


location_counts = to_use["location"][to_use["type"] == "Provoked"].value_counts() #same thing as below
location_frequencies = to_use["location"][to_use["type"] == "Provoked"].value_counts().to_dict() # but this one returns a dictionary 
    

In [3]:
#this country count is useful only if we have country preferences 
country_counts = to_use["country"][to_use["type"] == "Provoked"].value_counts()
country_frequencies = to_use["country"][to_use["type"] == "Provoked"].value_counts().to_dict()

NameError: name 'to_use' is not defined

In [ ]:
species_counts = to_use["species"][to_use["type"] == "Provoked"].value_counts()
species_counts

# we were doing this earlier - but it is wrong - because it is species to type but not to location
species_counts =to_use["species"][to_use["type"] == "Provoked"].value_counts()
species_frequencies = to_use["species"][to_use["type"] == "Provoked"].value_counts().to_dict()

# we could choose one or could choose multiple - i've done the count for all - location_counts
# does this make sense? because provoked to species will give the most frequently occurring provoked attacks by type of sharks worldwide 
# we want location specific species

In [ ]:
#so now i will do a research on each #check word doc
#in key largo monroe county
#check if this is the same for you
species_loc_freq = to_use["species"][to_use["location"] == "Key Largo Monroe County"].value_counts().to_dict()
species_loc_freq



In [ ]:
#in blue planet aquarium ellesmere port"
species_loc_freq = to_use["species"][to_use["location"] == "Blue Planet Aquarium Ellesmere Port"].value_counts().to_dict()
species_loc_freq 

In [ ]:
#in puerto escondido
species_loc_freq = to_use["species"][to_use["location"] == "Puerto Escondido"].value_counts().to_dict()
species_loc_freq 

In [ ]:
#lauderdale by the Sea Broward County (this is a hyphenated name anyway officially)
species_loc_freq = to_use["species"][to_use["location"] == "LauderdalebytheSea Broward County"].value_counts().to_dict()
species_loc_freq 

In [ ]:
#New Croker Island"
species_loc_freq = to_use["species"][to_use["location"] == "Near Croker Island"].value_counts().to_dict()
species_loc_freq 